# Programming for Data Analysis Project 2020

## Problem Statement

- Choose a real-world phenomenon that can be measured and for which you could
collect at least one-hundred data points across at least four different variables.

- Investigate the types of variables involved, their likely distributions, and their relationships with each other.

- Synthesise/simulate a data set as closely matching their properties as possible.

- Detail your research and implement the simulation in a Jupyter notebook – the
data set itself can simply be displayed in an output cell within the notebook.


### Project Outline

Subject Matter - Road Collisions for a given year. For this project we will use 2018. To create a dataset, intial variables are:
1. Collision type
2. Month
3. Day
4. Date
5. Month
6. Time Band
7. Sex
8. Age
9. Road User.
10. County (maybe)

In 2018 there was 8150 fatal and serious casulaties so this will be our sample size for the intial dataset. Once this is complete another summary dataset can be created summarised by date. In this table we may add, Speeding fines, traffic volumes - km driven for further analyitical purposes.

### Project Plan

#### 1. Gathering research data
* Gather data from CSO datasets and RSA datsets for collisions by:
  - Month
  - Day
  - Time Band
  - Road User (cover Sex and Age)

* Traffic data from the annual Transport Omnibus 

* Fixed Charged Notices from Garda.ie for
  - Speeding
  - Intoxication
  - Mobile Phones
  
Initial formatting of csv files in order to create Pandas dataframes

#### 2. EDA (Research) of these datasets
* To understand required distributions, averages etc. to create out dataset
  - 5 years of data for percentage breakdowns, averages and weighted averages not for overall figures.

#### 3. Variables and Functions
* Create relevant variables and functions to be used for simulated dataset
  - p values
  - weighted averages
  - lists
  - dictionaries
  
#### 4. Create intial dataset
* creation of categorical based dataset that summary will be based on.
  - Create dates based on monthly and daily p values and distribution
  - dates will automatically created day and month
  - Monthly collisions by type (conditional binomial by month depending on EDA)
  - Time band - might be by month depending on initial EDA
  - Road User - might be by month depending on initial EDA
  - Sex (conditional binomial by month depending on EDA)

#### 5. Summary datset
* summarise intial dataset by data
* add in traffic and fcns

#### 6. EDA of the summary Dataset.
* summarise by each variable
* pairplot for collisons, fcns, traffic





### Python Libaries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

rng = np.random.default_rng

## Road Collisions on Irish Roads



In [4]:
df = pd.read_csv('by_Month.csv')

In [5]:
df

,Month,Type,2014,2015,2016,2017,2018
0,January,Fatalities,16,12,15,16,12
1,February,Fatalities,13,13,10,11,10
2,March,Fatalities,15,11,17,20,14
3,April,Fatalities,14,7,14,7,17
4,May,Fatalities,18,12,20,8,4
5,June,Fatalities,18,13,12,14,16
6,July,Fatalities,18,19,21,16,8
7,August,Fatalities,15,14,13,10,12
8,September,Fatalities,11,16,18,13,10
9,October,Fatalities,17,11,20,11,9


In [10]:
df.groupby(df['Month']).sum()

,2014,2015,2016,2017,2018
Month,,,,,
April,656,542,544,543,619
August,756,690,639,645,646
December,709,747,694,732,720
February,590,569,620,533,585
January,652,690,711,592,610
July,703,686,743,671,738
June,698,659,654,637,722
March,590,654,614,692,609
May,671,702,642,712,737


In [17]:
df['Month'][df['Type']=='Serious'].sum()

'JanuaryFebruaryMarchAprilMayJuneJulyAugustSeptemberOctoberNovemberDecember'